# Process bulk profiles

## Import libraries

In [1]:
import pathlib
import pprint

import pandas as pd

from pycytominer import aggregate, annotate, normalize, feature_select
from pycytominer.cyto_utils import output

## Set paths and variables

In [2]:
# Path to dir with cleaned data from single-cell QC
cleaned_dir = pathlib.Path("./data/cleaned_profiles")

# output path for bulk profiles
output_dir = pathlib.Path("./data/bulk_profiles")
output_dir.mkdir(parents=True, exist_ok=True)

# extract the plate names from the file name
plate_names = [file.stem.split("_")[0] for file in cleaned_dir.glob("*.parquet")]

# path for platemap directory
platemap_dir = pathlib.Path("../0.download_data/metadata/platemaps")

# load in barcode platemap
barcode_platemap = pd.read_csv(
    pathlib.Path(f"{platemap_dir}/Barcode_platemap_pilot_data.csv")
)

# operations to perform for feature selection
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
    "drop_na_columns",
]

plate_names

['BR00143980',
 'BR00143977',
 'BR00143981',
 'BR00143978',
 'BR00143979',
 'BR00143976']

## Set dictionary with plates to process

In [3]:
# create plate info dictionary
plate_info_dictionary = {
    name: {
        "profile_path": (
            str(
                pathlib.Path(
                    list(cleaned_dir.rglob(f"{name}_cleaned.parquet"))[0]
                ).resolve(strict=True)
            )
            if list(cleaned_dir.rglob(f"{name}_cleaned.parquet"))
            else None
        ),
        # Find the platemap file based on barcode match and append .csv
        "platemap_path": (
            str(
                pathlib.Path(
                    list(
                        platemap_dir.rglob(
                            f"{barcode_platemap.loc[barcode_platemap['barcode'] == name, 'platemap_file'].values[0]}.csv"
                        )
                    )[0]
                ).resolve(strict=True)
            )
            if name in barcode_platemap["barcode"].values
            else None
        ),
        # Get the time_point based on the barcode match
        "time_point": (
            barcode_platemap.loc[
                barcode_platemap["barcode"] == name, "time_point"
            ].values[0]
            if name in barcode_platemap["barcode"].values
            else None
        ),
    }
    for name in plate_names
}

# Display the dictionary to verify the entries
pprint.pprint(plate_info_dictionary, indent=4)

{   'BR00143976': {   'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate1_platemap.csv',
                      'profile_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/cleaned_profiles/BR00143976_cleaned.parquet',
                      'time_point': 24},
    'BR00143977': {   'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate2_platemap.csv',
                      'profile_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/cleaned_profiles/BR00143977_cleaned.parquet',
                      'time_point': 24},
    'BR00143978': {   'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate1_platemap.csv',
                      'profile_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/cleaned_profiles/BR00143978_cleaned.parq

## Process data with pycytominer

In [4]:
for plate, info in plate_info_dictionary.items():
    print(f"Now performing pycytominer pipeline for {plate}")

    # Output file paths
    output_aggregated_file = str(pathlib.Path(f"{output_dir}/{plate}_bulk.parquet"))
    output_annotated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_annotated.parquet")
    )
    output_normalized_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_normalized.parquet")
    )
    output_feature_select_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_feature_selected.parquet")
    )

    # Load single-cell profiles
    single_cell_df = pd.read_parquet(info["profile_path"])

    # Load platemap
    platemap_df = pd.read_csv(info["platemap_path"])

    # Step 1: Aggregation
    aggregate(
        population_df=single_cell_df,
        operation="median",
        strata=["Image_Metadata_Plate", "Image_Metadata_Well"],
        output_file=output_aggregated_file,
        output_type="parquet",
    )

    # Step 2: Annotation
    annotated_df = annotate(
        profiles=output_aggregated_file,
        platemap=platemap_df,
        join_on=["Metadata_well", "Image_Metadata_Well"],
    )

    # Step 2.5: Add 'Metadata_time_point' column based on the plate's time_point from dict
    annotated_df["Metadata_time_point"] = info["time_point"]

    # Step 3: Output annotated DataFrame
    output(
        df=annotated_df,
        output_filename=output_annotated_file,
        output_type="parquet",
    )

    # Step 4: Normalization
    normalize(
        profiles=annotated_df,
        method="standardize",
        output_file=output_normalized_file,
        output_type="parquet",
    )

    # Step 5: Feature selection
    feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        na_cutoff=0,
        output_file=output_feature_select_file,
        output_type="parquet",
    )

Now performing pycytominer pipeline for BR00143980
Now performing pycytominer pipeline for BR00143977
Now performing pycytominer pipeline for BR00143981
Now performing pycytominer pipeline for BR00143978
Now performing pycytominer pipeline for BR00143979
Now performing pycytominer pipeline for BR00143976


In [5]:
# Check output file
test_df = pd.read_parquet(output_feature_select_file)

print(test_df.shape)
test_df.head(2)

(240, 453)


,Metadata_cell_line,Metadata_row,Metadata_column,Metadata_seeding_density,Metadata_Plate,Metadata_Well,Metadata_time_point,Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,...,Nuclei_Texture_InfoMeas1_CorrAGP_3_00_256,Nuclei_Texture_InfoMeas1_CorrAGP_3_01_256,Nuclei_Texture_InfoMeas1_CorrER_3_00_256,Nuclei_Texture_InfoMeas1_CorrMito_3_02_256,Nuclei_Texture_InfoMeas2_CorrAGP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CorrBrightfield_3_03_256,Nuclei_Texture_InverseDifferenceMoment_CorrMito_3_01_256,Nuclei_Texture_SumEntropy_CorrDNA_3_01_256,Nuclei_Texture_SumEntropy_CorrMito_3_00_256,Nuclei_Texture_SumVariance_CorrBrightfield_3_02_256
0,DAOY,C,3,1000,BR00143976,C03,24,-0.559133,-1.750296,2.156141,...,-1.333499,-0.630385,0.014052,-0.033966,0.253926,0.705096,-0.508078,0.245258,1.303425,-0.685422
1,DAOY,D,3,1000,BR00143976,D03,24,-0.736887,-1.544902,1.767861,...,-1.055713,-0.617657,0.163833,0.159932,0.289750,0.385109,-0.178750,0.332079,0.928611,-0.638371
